# The JuMP ecosystem for mathematical optimization: MOI

## JuliaCon 2018

## Juan Pablo Vielma
## MIT Sloan

In [ ]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

## MOI and JuMP

In [ ]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

Behind every JuMP model there is a MOI model (used to be a MPB model)

In [ ]:
typeof(model.moibackend)

# MOI and JuMP variables

```julia
struct JuMP.VariableRef # MOI variable + owning model
    m::Model
    index::MOI.VariableIndex
end

struct MOI.VariableIndex  # Type-safe unique non-consecutive index
    value::Int64
end
```

In [ ]:
xx = MOI.get(model.moibackend,MOI.ListOfVariableIndices()) 

In [ ]:
x[1].index == xx[1] && x[1].m == model

# MOI and JuMP constraints: First MOI constraints

In [ ]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone()) 

L,Q,C = MOI.get(model.moibackend,MOI.ListOfConstraints())

# MOI constraints format 
$$
f(x) \in S
$$

were $f$ is an ``MOI.AbstractFunction`` and $S$ is an ``MOI.AbstractSet`` 
## Example:
JuMP:
```julia
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)
```
``MOI.ScalarAffineFunction``-in-``MOI.LessThan``:

$$a^T x + b \in \{y:y\leq u\} \text{ for } a=(2,1)^T,\text{ }b=0 \text{ and }u=0$$ 

# More Constraint Examples

| Mathematical Constraint       | MOI Function                 | MOI Set        |
|-------------------------------|------------------------------|----------------|
| $a^Tx = b$                  | `ScalarAffineFunction`       | `EqualTo`      |
| $l \le a^Tx \le u$          | `ScalarAffineFunction`       | `Interval`     |
| $x_i \le u$                 | `SingleVariable`             | `LessThan`     |
| $Ax + b \in \mathbb{R}_+^n$ | `VectorAffineFunction`       | `Nonnegatives` |
| $\lVert Ax + b\rVert_2 \le c^Tx + d$                        | `VectorAffineFunction`       | `SecondOrderCone` |
| $y \exp (x/y) \le z, y > 0$  | `VectorOfVariables`       | `ExponentialCone`                  |
| $x \in \mathbb{R}^{d\left(d+1\right)/2}$,◹$(x)\in \text{PSD} \subseteq \mathbb{R}^{d\times d}$                                       | `VectorOfVariables`          | `PositiveSemidefiniteConeTriangle` |
| $x^TQx + a^Tx + b \ge 0$    | `ScalarQuadraticFunction`    | `GreaterThan`                 |
| $x_i \in \mathbb{Z}$                                                                     | `SingleVariable`    | `Integer`                          |
| $x_i \in \{0,1\}$                                                                        | `SingleVariable`    | `ZeroOne`                          |
| $x_i \in \{0\} \cup [l,u]$                                                               | `SingleVariable`    | `Semicontinuous`                   |
| At most one component of $x$ can be nonzero                                              | `VectorOfVariables` | `SOS1`                             |

MOI includes standard functions and sets, and allows for some extensions. See [MOI manual](http://www.juliaopt.org/MathOptInterface.jl/stable/index.html) for more details on [supported sets and functions](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Standard-form-problem-1) and more [constraint examples](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Constraints-by-function-set-pairs-1).

# MOI and JuMP constraints

```julia
struct JuMP.ConstraintRef{C} # MOI constraint + owning model
    m::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end

struct MOI.ConstraintIndex{F,S} # Type-safe unique (within class) non-consecutive  index for class F-in-S
    value::Int64
end
```

```julia
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone()) 
```

In [ ]:
linear = MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{L...}())

# Example: Linear inequalities

JuMP:
```julia
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)
```
``MOI.ScalarAffineFunction``-in-``MOI.LessThan``:

$$a^T x + b \in \{y:y\leq u\} \text{ for } a=(2,1)^T,\text{ }b=0 \text{ and }u=0$$ 

In [ ]:
linear1.index == linear[1] && linear1.m == model

$a=(2,1)^T$:

In [ ]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).terms

$b=0$:

In [ ]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).constant

$u=0$:

In [ ]:
MOI.get(model.moibackend,MOI.ConstraintSet(),linear[1]).upper

## Similar for other constraints:

In [ ]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarQuadraticFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{Q...}())

In [ ]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.VectorAffineFunction{Float64}, MOI.SecondOrderCone}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{C...}())

## MOI Constraints

### Note: there are multiple ways to write the same constraint. For more details and discussions see [Constraint Bridges](#Constraint-Bridges).

## A Complete MOI Model

In [ ]:
using MathOptInterface
const MOI = MathOptInterface
using GLPK

# Solve the binary-constrained knapsack problem: max c'x: w'x <= C, x binary using GLPK.

c = [1.0, 2.0, 3.0]
w = [0.3, 0.5, 1.0]
C = 3.2

numvariables = length(c)

optimizer = GLPK.GLPKOptimizerMIP()

# create the variables in the problem
x = MOI.addvariables!(optimizer, numvariables)

# set the objective function
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set!(optimizer, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(), objective_function)
MOI.set!(optimizer, MOI.ObjectiveSense(), MOI.MaxSense)

# add the knapsack constraint
knapsack_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0)
MOI.addconstraint!(optimizer, knapsack_function, MOI.LessThan(C))

# add integrality constraints
for i in 1:numvariables
    MOI.addconstraint!(optimizer, MOI.SingleVariable(x[i]), MOI.ZeroOne())
end

# all set
MOI.optimize!(optimizer)

termination_status = MOI.get(optimizer, MOI.TerminationStatus())
objvalue = MOI.canget(optimizer, MOI.ObjectiveValue()) ? MOI.get(optimizer, MOI.ObjectiveValue()) : NaN
if termination_status != MOI.Success
    error("Solver terminated with status $termination_status")
end

@assert MOI.get(optimizer, MOI.ResultCount()) > 0

result_status = MOI.get(optimizer, MOI.PrimalStatus())
if result_status != MOI.FeasiblePoint
    error("Solver ran successfully did not return a feasible point. The problem may be infeasible.")
end
primal_variable_result = MOI.get(optimizer, MOI.VariablePrimal(), x)

@show objvalue
@show primal_variable_result

## Model modifications

You can delete variables or constraints, modify constraints or transform constraints. Modifications are as efficient as allowed by the solver.

Example: Transform constraint

In [ ]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

In [ ]:
rsocp = MOI.transform!(model.moibackend, socp.index, MOI.RotatedSecondOrderCone(3))

In [ ]:
rsocp != socp

In [ ]:
MOI.isvalid(model.moibackend,socp.index)

In [ ]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, linear1, 2 * x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, linear2, 2 * x[1] + 2 * x[2] <= 1) 
@constraint(model, quadratic, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

Delete constraint:

In [ ]:
if MOI.candelete(model.moibackend,x[1].index)
    MOI.delete!(model.moibackend,x[1].index)
end
if MOI.candelete(model.moibackend,linear1.index)
    MOI.delete!(model.moibackend,linear1.index)
end

Modify constraints:

In [ ]:
if MOI.canmodify(model.moibackend, typeof(linear2.index), MOI.ScalarCoefficientChange{Float64})
    MOI.modify!(model.moibackend, linear2.index, MOI.ScalarCoefficientChange(x[1].index, 3.0))
end

## MOI Attributes: For MOI models

In [ ]:
model = GLPK.GLPKOptimizerMIP()
var_index = MOI.addvariable!(model)
constr_index = MOI.addconstraint!(model,MOI.SingleVariable(var_index),MOI.LessThan(10.0))

for building models...

In [ ]:
MOI.set!(model,MOI.ObjectiveSense(),MOI.MaxSense)
MOI.set!(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), MOI.SingleVariable(var_index))

and more...

In [ ]:
if MOI.canset(model,MOI.VariablePrimalStart(),typeof(var_index))
    MOI.set!(model,MOI.VariablePrimalStart(),var_index,5.0)
end
if MOI.canset(model,MOI.ConstraintDualStart(),typeof(constr_index))
    MOI.set!(model,MOI.ConstraintDualStart(),constr_index,5.0)
end

## MOI Attributes: Also for JuMP models

In [ ]:
model = Model()

some attributes have custom macro syntax

In [ ]:
@variable(model, x >= 10.0, start = 5.0)
@objective(model, Max, x)

others use MOI functions

In [ ]:
bound_ref = JuMP.LowerBoundRef(x).index
if MOI.canset(model.moibackend,MOI.ConstraintDualStart(),typeof(bound_ref))
    MOI.set!(model.moibackend,MOI.ConstraintDualStart(),bound_ref,10.0)
end

## JuMP and MOI Solver Modes 

In [ ]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model, inequality, x + y <= 1)
@objective(model, Max, x + y)

JuMP has 3 solver modes:

In [ ]:
JuMP.ModelMode

Default mode is `Automatic`

In [ ]:
JuMP.mode(model) == JuMP.Automatic

In [ ]:
c = [1; 3; 5; 2] 

A= [
     1 1 9 5;
     3 5 0 8;
     2 0 6 13
    ]

b = [7; 3; 5] 

m, n = size(A); # m = number of rows of A, n = number of columns of A

# One Extreme: Automatic Mode = "Almost" Old JuMP

`CachingOptimizer` re-builds model when incremental modifications are not valid.
* Pros: No need to worry about what modifications are valid.
* Cons: No information about efficiency of modifications (may be re-creating the whole model for each modification)

**Note:** Automatic mode also provides automatic constraint transformations. See [Constraint Bridges](#Constraint-Bridges)

In [ ]:
model = JuMP.Model(with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 0))
@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 
@time begin
JuMP.optimize(model) # solves the model
end
@show JuMP.terminationstatus(model) == MOI.Success

**Note:** 
```julia
with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 0)
```
builds a factory that can create instances of 
```julia
GLPK.GLPKOptimizerLP(msg_lev = 0)
```
to re-build the internal model (or copy it).

### The Other Extreme: Direct Mode

Direct connection to a solver. All data is stored with solver, MOI does not save or cache anything.
* Pros: Access to complete solver interfase (e.g. callbacks). Allowed modifications are as efficient as possible for the solver. 
* Cons: Cannot apply incremental modifications not supported by the solver. Cannot copy the mode. Cannot use [Constraint Bridges](#Constraint-Bridges)

In [ ]:
model = JuMP.direct_model(GLPK.GLPKOptimizerLP(msg_lev = 0))
@variable(model, x[1:n] >= 0) # Models x >=0
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 
@time begin
JuMP.optimize(model) # solves the model
end
@show JuMP.terminationstatus(model) == MOI.Success

### Intermediate Mode: Manual Mode = Direct Control of `CachingOptimizer`

In [ ]:
model = JuMP.Model(with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 4),caching_mode = MOIU.Manual)

@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

JuMP.optimize(model)

Need to control `CachingOptimizer` directly.

 # Controlling `CachingOptimizer` 

In [ ]:
copt = JuMP.caching_optimizer(model)
typeof(copt)

In [ ]:
MOIU.CachingOptimizerState

In [ ]:
MOIU.state(copt) == MOIU.EmptyOptimizer

`EmptyOptimizer` has not loaded the model. We can load it with `attachoptimizer!` and then we can solve:

In [ ]:
MOIU.attachoptimizer!(copt);

In [ ]:
MOIU.state(copt) == MOIU.AttachedOptimizer

In [ ]:
JuMP.optimize(model)

Unsuported incremental modifications through an error for `AttachedOptimizer`, but not for `EmptyOptimizer`.
We can return to `EmptyOptimizer` with `resetoptimizer`:

In [ ]:
MOIU.resetoptimizer!(copt);
MOIU.state(copt) == MOIU.EmptyOptimizer

### Optional Cbc example

In [ ]:
using Cbc
model = JuMP.Model(with_optimizer(Cbc.CbcOptimizer),caching_mode = MOIU.Manual)

@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

In [ ]:
copt = JuMP.caching_optimizer(model)
MOIU.attachoptimizer!(copt);
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success

In [ ]:
@constraint(model, x[1]==1)

In [ ]:
MOIU.resetoptimizer!(copt);
@constraint(model, x[1]==1)
MOIU.attachoptimizer!(copt);
JuMP.optimize(model)

# Constraint Bridges

## Constraint Bridges: Example 1

In [ ]:
model = JuMP.Model()
@variable(model,x[1:10]);

Two-sided linear constraint as one `ScalarAffineFunction`-in-`Interval`:

In [ ]:
@constraint(model, interval_inequality, -2 <= sum(x) <= 1)
#interval_inequality.index

Two-sided linear constraint as two `ScalarAffineFunction`-in-`LessThan`:

In [ ]:
@constraint(model, lower_bound, -sum(x) <= 2)
#lower_bound.index
@constraint(model, upper_bound,  sum(x) <= 1)
#upper_bound.index

## Constraint Bridges: Example 2

In [ ]:
model = JuMP.Model()
@variable(model,x)
@variable(model,y)
@variable(model,t);

Quadratic inequality as `ScalarQuadraticFunction`-in-`LessThan`:

In [ ]:
@constraint(model,quadratic, x^2 + y^2 <= t)
#quadratic.index

Quadratic inequality as `VectorAffineFunction`-in-`SecondOrderCone`:

In [ ]:
@constraint(model, conic, [(1+t)/2; (1-t)/2; x; y] in SecondOrderCone())
conic.index

# [Constraint Bridges](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Constraint-bridges-1) provide automatic transformation rules.

For instance, [Automatic Mode](#One-Extreme:-Automatic-Mode-=-"Almost"-Old-JuMP) actually adds two intermadiate layers: 
1. `CachingOptimizer` for incremental modifications, and
2. `LazyBridgeOptimizer <: AbstractBridgeOptimizer` for default automatic transformations

`LazyBridgeOptimizer` can be disabled with the `bridge_constraints = false` keyword argument.

# Example for `bridge_constraints`

ECOS does not support quadratic constraints so it fails without a bridge

In [ ]:
using ECOS
model = JuMP.Model(with_optimizer(ECOS.ECOSOptimizer), bridge_constraints = false)
@variable(model,x)
@variable(model,y)
@variable(model,t)
@constraint(model,quadratic, x^2 + y^2 <= t)

It will work once the standard bridge is [implemented](https://github.com/JuliaOpt/MathOptInterfaceBridges.jl/issues/93)

In [ ]:
model = JuMP.Model(with_optimizer(ECOS.ECOSOptimizer)) #, bridge_constraints = true)
@variable(model,x)
@variable(model,y)
@variable(model,t)
@constraint(model,quadratic, x^2 + y^2 <= t)

** Note: ** Bridges also should transform dual solutions.

## JuMP and MOI

Note: Need the following patch [implemented in MOI master](https://github.com/JuliaOpt/MathOptInterface.jl/pull/409), but not on release v0.4.1

In [ ]:
MOI.canget(m::MOIU.CachingOptimizer, IdxT::Type{<:MOI.Index}, name::String) = MOI.canget(m.model_cache, IdxT, name)
MOI.get(m::MOIU.CachingOptimizer, IdxT::Type{<:MOI.Index}, name::String) = MOI.get(m.model_cache, IdxT, name)

In [ ]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model, inequality, x + y <= 1)
@objective(model, Max, x + y)

In [ ]:
#@macroexpand @variable(model, x >= 0)

`@variable(model, x >= 0)`:

1. Create JuMP/MOI variable and set its attributes:
```julia
x = JuMP.addvariable(model,[...],"x")
```
2. Save JuMP variable in symbol-keyed dictionary:
```julia
JuMP.registervar(model, :x, x)
```

In [ ]:
#@macroexpand @constraint(model, inequality, x + y <= 1)

`@constraint(model, inequality, x + y <= 1)`:

1. Create JuMP/MOI constraint (function and set):
```julia
inequality = JuMP.addconstraint(model,[...],"inequality")
``` 
2. Save JuMP constraint in symbol-keyed dictionary:
```julia
JuMP.registercon(model, :inequality, inequality)
``` 

#### JuMP stores data only on MOI

```julia
function addconstraint(m::Model, c::AbstractConstraint, name::String="")
    f, s = moi_function_and_set(c)
    [...]
    cindex = MOI.addconstraint!(m.moibackend, f, s)::ConstraintIndex{F,S} 
    cref = ConstraintRef(m, cindex)::ConstraintRef{ConstraintIndex{F,S}}
    if !isempty(name)
        MOI.set!(m.moibackend, MOI.ConstraintName(), cindex,name)
    end
    return cref
end
```
JuMP constraint:
```julia
struct ConstraintRef{C}
    m::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end
```

We can recover MOI constraints and variables by their "name":

In [ ]:
MOI.get(model.moibackend, MOI.ConstraintIndex, "inequality")  == inequality.index

In [ ]:
MOI.get(model.moibackend, MOI.VariableIndex, "x")  == x.index

and get the name of a constraint or variable:

In [ ]:
MOI.get(model.moibackend, MOI.ConstraintName(),inequality.index)

In [ ]:
MOI.get(model.moibackend, MOI.VariableName(),x.index)

Note: Simpler syntax to get name of JuMP variable will be available by [v0.19 release](https://github.com/JuliaOpt/JuMP.jl/issues/1184) 

#### JuMP does store dictionary to recover variables/constraints by symbol

In [ ]:
model[:x] == x

In [ ]:
model[:inequality] == inequality

usefull when working with multiple models with same names:

In [ ]:
model1 = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model1, x >= 0)
@variable(model1, y >= 0)
@constraint(model1, inequality, x + y <= 1)
@objective(model1, Max, x + y)

model2 = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model2, x >= 0)
@variable(model2, y >= 0)
@constraint(model2, inequality, 2x + y <= 1)
@objective(model2, Max, x + y)

`inequality` for `model1` was overwritten by `inequality` for `model2`: 

In [ ]:
inequality

we can recover `inequality` for `model1` from `model1[...]`:

In [ ]:
model1[:inequality]

### A recap of the four types of variables/constraints in JuMP

Julia variables v/s mathematical optimization variables?! 

In [ ]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x[1:3] >= 0, basename="Xvar")
@variable(model, y >= 0)
@constraint(model, inequality, sum(x) + y <= 1)
@objective(model, Max, sum(x) + y)

MOI variable: a unique index within a model

In [ ]:
y.index

In [ ]:
x[2].index

MOI variable name: a unique string associated to a MOI variable.

In [ ]:
MOI.get(model.moibackend, MOI.VariableName(),y.index)

In [ ]:
MOI.get(model.moibackend, MOI.VariableName(),x[2].index)

In [ ]:
MOI.set!(model.moibackend, MOI.VariableName(),x[3].index,"lastXvar")

In [ ]:
x

Single JuMP variable: Reference to a MOI variable in a model. A Julia variable of type `JuMP.VariableRef`

In [ ]:
typeof(y)

In [ ]:
y.index

In [ ]:
y.m == model

Container of JuMP variables: well... a container of `JuMP.VariableRef` 

In [ ]:
typeof(x)

### Interesting Examples:

In [ ]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
x = @variable(model, [1:3])

In [ ]:
x

In [ ]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, [1:3], basename=string(rand()))

In [ ]:
using DataStructures
myvariables = Stack(JuMP.VariableRef)
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
push!(myvariables,@variable(model, y >= 0))
push!.(myvariables,@variable(model, z[1:3] >= 0))
pop!(myvariables)

In [ ]:
@show myvariables